In [1]:
#import necessary packages
import json
import pandas as pd
from textblob import TextBlob
import time

#Read in both data frames
#Set filename and read through review JSON file line by line
if False:
    filename_review = "/Users/andresbaez/Desktop/COGS108_Project/yelp_dataset/review.json"
    #Create dataframe that will contain review data
    df_review = pd.DataFrame()
    #go through each JSON object and create a dictionary out of its key-value pairs
    st = time.time()
    with open(filename_review,'r') as f:
        for line in f:
            data = json.loads(line)
            #append each JSON object to df_review
            df_review = df_review.append(data, ignore_index=True)       
            if (time.time() - st) > 60:
                break
    df_review.to_csv("/Users/andresbaez/Desktop/COGS108_Project/yelp_dataset/review.csv")
    #Set filename and read through user JSON file line by line
    filename_users = "/Users/andresbaez/Desktop/COGS108_Project/yelp_dataset/user.json"
    #Create dataframe that will contain user data
    df_users = pd.DataFrame()
    #go through each JSON object and create a dictionary out of its key-value pairs
    st = time.time()
    with open(filename_users,'r') as f:
        for line in f:
            data = json.loads(line)
            #append each JSON object to df_users
            df_users = df_users.append(data, ignore_index=True)
            if (time.time() - st) > 60:
                break
    df_users.to_csv("/Users/andresbaez/Desktop/COGS108_Project/yelp_dataset/user.csv")


filename_review = "/Users/andresbaez/Desktop/COGS108_Project/yelp_dataset/review.json"
#Create dataframe that will contain review data
df_review = pd.DataFrame()
#go through each JSON object and create a dictionary out of its key-value pairs
st = time.time()
with open(filename_review,'r') as f:
    for line in f:
        data = json.loads(line)
        #append each JSON object to df_review
        df_review = df_review.append(data, ignore_index=True)       
        if (time.time() - st) > 300:
              break
df_review.to_csv("/Users/andresbaez/Desktop/COGS108_Project/yelp_dataset/review2.csv")

In [3]:
#Read newly exported CSV files into dataframe
df_review = pd.read_csv('./yelp_dataset/review2.csv')
df_user = pd.read_csv('./yelp_dataset/user.csv')

#Start out by dropping columns that are not necessary to our analysis
df_review.drop(['business_id','cool','funny','useful'],axis=1,inplace=True)
df_user.drop(['friends','compliment_cool','compliment_cute','compliment_funny','compliment_hot','compliment_list',
              'compliment_more','compliment_note','compliment_photos','compliment_writer','compliment_plain','compliment_profile',
              'cool','fans','funny','name','useful'],axis=1,inplace=True)
#Want to drop any columns where the user has never been an elite member
df_user.dropna(subset = ['elite'],inplace=True)

#Want to merge our user and review data now that we've dropped unnecessary rows and columns
df = pd.merge(df_user,df_review,on='user_id',how='right')
df_review.shape

(17288, 6)

In [ ]:
#Want to parse out year from date (YEAR-DAY-MONTH)
def getYear(date):
    year = date.split("-")[0]
    return year

#assumes stars is int and reviewText is string
def find_sentiment(stars, reviewText):
    review = TextBlob(reviewText)
    
    #convert review sentiment from [-1,1] to [0,5]
    reviewScore = (review.sentiment[0] + 1) / 2
    reviewScore = reviewScore * 5
    
    #return average of scores
    score = (stars + reviewScore) / 2
    return score

def isElite(year, user_id):
    if year in df['elite']:
        return True
    else:
        return False

#Apply function to obtain year from date in both dataframes
df_review['year'] = df_review['date'].apply(getYear)
df_user['year'] = df_user['yelping_since'].apply(getYear)

#Apply function to find if user was an Elite member at the time of the review
df_review['Elite_Reviewer'] = df_review.apply(lambda x: isElite(df_review['year'][0], df_review['user_id'][0]), axis=1)

In [ ]:
#Apply function to find sentiment for each review in df_review
#df_review['critical_index'] = df_review.apply(lambda x: find_sentiment(df_review['stars'], df_review['text'][0]), axis=1)
